In [ ]:
import numpy as np
import plotly.graph_objects as go
import hyperspy.api as hs
from plotly.subplots import make_subplots

In [ ]:
path = "../../../Masteroppgave/2023-03-08_EDS-Apreo/exports/"
s = hs.load(path + 'GaAs_30kV_50pA.emsa', signal_type='EDS_SEM')
elements = ['Ga', 'As']
lines = ['Ga_Ll','Ga_La', 'As_La',  'Ga_Ka', 'Ga_Kb', 'As_Ka', 'As_Kb']
names = ['Ga Ll','Ga Lα', 'As Lα', 'Ga Kα', 'Ga Kβ',  'As Kα', 'As Kβ']
s.set_elements(elements)
s.data[:33] = 0
m = s.create_model()
m.fit()
scale = s.axes_manager[0].scale
offset = s.axes_manager[0].offset
kev = s.axes_manager[0].axis
x = kev
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'] # plotly colors

In [ ]:
def kev2channel(kev):
    return (kev - offset) / scale
def channel2kev(channel):
    return channel * scale + offset
def remove_low_values(y, threshold):
    y[y < threshold] = np.nan
    return y
def theoretical_energy(line, what='energy (keV)'): # what can be 'energy (keV)' or 'weight'
    return hs.material.elements[line.split('_')[0]]['Atomic_properties']['Xray_lines'][line.split('_')[1]][what]

In [ ]:
def gaussian2(x, x0, fwhm, area):
    fwhm = fwhm / 1000  # eV to keV
    sigma = fwhm / 2.35482
    return area / (sigma * np.sqrt(2 * np.pi)) * np.exp(-((x - x0) / sigma)**2 / 2)
def fwhm2sigma(fwhm):
    return fwhm / (2 * np.sqrt(2 * np.log(2)))

In [ ]:
fig = go.Figure()
fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.99))
fig.update_layout(width=800,height=500,)
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))

fig.update_xaxes(title='Energy [keV]', range=[5.55,6.25])
fig.update_yaxes(title='Intensity [counts]', range=[-1500, 3.5e4])

fwhms = [120, 130, 160, 200]
e = 5.8987
area = 4000
g = []
for fwhm in fwhms:
    g.append(gaussian2(x, e, fwhm, area))
    fig.add_trace(go.Scatter(x=x, y=g[-1]+1000, name=f'FWHM = {fwhm} eV', mode='lines', line_width=3))

fig.add_vline(x=e, line_width=2, line_color="Grey")#, line_dash="dash")
fig.add_annotation(x=e-.09, y=3.3e4, xref="x", yref="y", text="Mn Kα, 5.8987 keV", showarrow=False, font=dict(size=16, color="Grey"))

g_max = np.max(g, axis=1)
for i in range(len(fwhms)):
    fig.add_trace(go.Scatter(x=[e-fwhms[i]/1000/2, e+fwhms[i]/1000/2], y=[g_max[i]/2, g_max[i]/2], mode='lines', line_width=2, line_color=colors[i], showlegend=False, name=f'FWHM = {fwhms[i]} eV', line_dash='dot'))

fig

In [ ]:
fwhms = [120, 130, 160, 200]
e = 5.8987
area = 4000
g = []

In [ ]:
fig = make_subplots(rows=1, cols=2, shared_xaxes=True, horizontal_spacing=0.06)
fig.update_layout(font=dict(family="EB Garamond SemiBold", size=16, color="black"))
fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.99))
fig.update_layout(width=1200,height=500,)
fig.update_layout(margin=dict(l=10, r=10, t=10, b=70))

# fig.update_xaxes(title='Energy [keV]', range=[5.55,6.25])
fig.update_xaxes(range=[5.55,6.25])

# fig.update_yaxes(title='(b)', range=[-1500, 3.5e4], row=1, col=2, title_standoff=0)
# fig.update_yaxes(type="log", range=[2.9, 4.65],title='Intensity [counts]<br>(a)', row=1, col=1, title_standoff=0)
fig.update_yaxes(range=[-1500, 3.5e4], row=1, col=2, title_standoff=0)
fig.update_yaxes(type="log", range=[2.9, 4.65],title='Intensity [counts]', row=1, col=1, title_standoff=0)
fig.update_xaxes(title='(a)', row=1, col=1, title_standoff=0)
fig.update_xaxes(title='(b)', row=1, col=2, title_standoff=0)


for fwhm in fwhms:
    g.append(gaussian2(x, e, fwhm, area))
    fig.add_trace(go.Scatter(x=x, y=g[-1]+1000, name=f'FWHM = {fwhm} eV', mode='lines', line_width=3, line_color=colors[fwhms.index(fwhm)]), row=1, col=2)
                            #  ), row=1, col=2)
    fig.add_trace(go.Scatter(x=x, y=g[-1]+1000, name=f'FWHM = {fwhm} eV', mode='lines', line_width=3, line_color=colors[fwhms.index(fwhm)], showlegend=False), row=1, col=1)

fig.add_vline(x=e, line_width=2, line_color="Grey")
# fig.add_annotation(x=e-.09, y=3.3e4, xref="x", yref="y", text="Mn Kα, 5.8987 keV", showarrow=False, font=dict(size=16, color="Grey"), row=1, col=1)
fig.add_annotation(x=e-.1, y=4.58, xref="x", yref="y", text="Mn Kα, 5.8987 keV", showarrow=False, font=dict(size=16, color="Grey"), row=1, col=1)


g_max = np.max(g, axis=1)
for i in range(len(fwhms)):
    fig.add_trace(go.Scatter(x=[e-fwhms[i]/1000/2, e+fwhms[i]/1000/2], y=[g_max[i]/2, g_max[i]/2], mode='lines', line_width=2, line_color=colors[i], 
                             showlegend=False, name=f'FWHM = {fwhms[i]} eV', line_dash='dot'), row=1, col=2)
    fig.add_trace(go.Scatter(x=[e-fwhms[i]/1000/2, e+fwhms[i]/1000/2], y=[g_max[i]/2, g_max[i]/2], mode='lines', line_width=2, line_color=colors[i], 
                             showlegend=False, name=f'FWHM = {fwhms[i]} eV', line_dash='dot'), row=1, col=1)

# setting a title at the bottom of the figure
fig.update_layout(title_text="Energy [keV]", title_x=0.52, title_y=0.01, title_font_size=16)

fig

In [ ]:
fig.write_image("../figures/energyResolutionsMnKa.pdf")
fig.write_image('../../mastersthesis/figures/eds_energyResolutionsMnKa.pdf')